<a href="https://colab.research.google.com/github/matthiasplum/CosmicRayML-Masterclass/blob/main/IceCube_Cosmic_Ray_Machine_Learning_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to a Cosmic Ray Module:
# Study Cosmic Rays with Machine Learning at the IceCube Neutrino Observatory

Cosmic rays include particles such as electrons, protons, and heavier nuclei. Their sources are not yet known, but we expect them to be both galactic and extragalactic. The energy spectrum of cosmic rays, which is how the rate changes with increasing energy, has been measured by many detectors, including IceTop. Extragalactic sources are expected to be more abundant as energy increases and to be the only ones responsible for very high-energy cosmic rays.

These cosmic messengers provide us with crucial insights into some of the most enigmatic phenomena in the universe, including the mysterious origins of cosmic rays themselves, the behavior of extreme astrophysical objects like black holes and supernovae, and the nature of dark matter and dark energy.

The IceCube Neutrino Observatory is an extraordinary facility located at the South Pole. It is a cubic kilometer of ice embedded with thousands of light sensors, buried deep beneath the Antarctic surface. Using these sensors, IceCube detects neutrinos, elusive subatomic particles produced by cosmic ray interactions, providing astronomers with a unique window into the high-energy universe. IceCube is complemented by the surface detector array IceTop, which utilizes an array of 1 square kilometer and measures cosmic rays in the energy range from about 100 TeV to 1 EeV, which happens to be the region of the transition from galactic to extragalactic cosmic rays.

In this module, we leverage the power of machine learning to delve deeper into the cosmic ray phenomena observed by IceCube and IceTop and showcase the methods to determine the mass composition of those particles.

 Machine learning, a branch of artificial intelligence, equips us with powerful tools to analyze complex datasets, recognize patterns, and make predictions. By applying machine learning techniques, we aim to uncover hidden insights, identify novel astrophysical sources, and enhance our understanding of cosmic ray physics.

**Only synthetic data are used in this module, but the analysis techniques are the ones we are using all the time.**

More information about the real analysis and the full scientific paper can be found here:

[Cosmic Ray Spectrum and Composition from PeV to EeV Using 3 Years of Data From IceTop and IceCube](https://arxiv.org/abs/1906.04317)


<p align="center">
<img src="https://masterclass.icecube.wisc.edu/wp-content/uploads/aboutcr-2.png" width="300" style="background-color: white;">
</p>
Energy spectrum of cosmic rays. The energy unit, the electronvolt (eV), is used by physicists to measure the energy and mass of relativistic particles. A typical chemical reaction happens in the eV energy regime, while nuclear reactions happen in the MeV (a million eV) range. The most powerful particle accelerator on Earth, the LHC, accelerates particles up to a few TeV (a million MeV). Cosmic rays reach energies above hundreds of EeV (a million TeV). Image: http://bit.ly/17gvx8C
<p align="center">
<img src="https://masterclass.icecube.wisc.edu/wp-content/uploads/measuring-shower.png" height="300" />
<img src="https://user-web.icecube.wisc.edu/~mplum/coincident.png" height="300" />
</p>
(left) A cosmic-ray shower reaches IceTop.
(right) A 'coincident' cosmic ray event was measured in IceTop and IceCube simultaniously.


In [ ]:
# @title Import necessary libraries and create list to store training results <font color='Red' size=4>ONLY RUN THIS ONCE!!!!</font>
# Import necessary libraries
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import powerlaw

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model ,to_categorical
from tensorflow.keras.callbacks import Callback

#This is used to save your training tries... ONLY RUN THIS ONCE!!!!
training_histories = []
training_features_histories = []

<font size=4>Create some particle detector capability specifications for your experiment.</font>

In [ ]:
#Specify Detector capabilities and energy spectrum index
energy_min = 10**6        #in GeV
energy_max = 10**9        #in GeV
zenith_min_deg = 0.       #in degree
zenith_max_deg = 60.      #in degree
spectral_index = -1.001     #Energy spectrum power law index

In [ ]:
# @title Load helper functions
#Define helper functions

class TimePerEpochCallback(Callback):
    def on_train_begin(self, logs=None):
        self.times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        self.times.append(epoch_time)
        if logs is not None:
            logs['time_per_epoch'] = epoch_time

def plot_loghist(x, bins, ax):
    hist, bins = np.histogram(x, bins=bins)
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    ax.hist(x, bins=logbins,alpha=0.5)
    ax.set_xscale('log')

def generate_power_law_data(num_events, energy_min, energy_max, spectral_index):
    # Generate uniform random numbers
    u = np.random.rand(num_events)

    # Convert uniform random numbers to follow power-law distribution
    energy = (energy_max**(spectral_index + 1) - energy_min**(spectral_index + 1)) * u + energy_min**(spectral_index + 1)
    energy = energy**(1/(spectral_index + 1))
    return energy

def gaussian_random_xmax(energy, std_deviation=1.0, slope=0.05, intercept=0.):
    # Calculate the mean as a linear function of energy
    E0 = 1e9  # Reference energy in GeV
    mean = intercept + slope*(np.log10(energy/E0)) # Example linear relation, adjust as needed
    # Generate a Gaussian random number with the calculated mean and provided standard deviation
    return np.random.normal(mean, std_deviation)

def gaussian_random_with_increasing_mean(energy, std_deviation=1.0, slope=0.05, intercept=0.):
    # Calculate the mean as a linear function of energy
    mean = intercept + slope*(np.log10(energy)) # Example linear relation, adjust as needed
    # Generate a Gaussian random number with the calculated mean and provided standard deviation
    return np.random.normal(mean, std_deviation)

def generate_cosmic_ray_labeled_data(n_events, particle_type):
    #Generate Type label
    ptype = [particle_type] * n_events

    # Generate energies following a power-law distribution
    energy = generate_power_law_data(n_events, energy_min, energy_max, spectral_index)

    # Generate zenith angles uniformly in arccos between zenith_min_deg and zenith_max_deg
    # Convert degrees to radians
    zenith_min_rad = np.radians(zenith_min_deg)
    zenith_max_rad = np.radians(zenith_max_deg)

    # Generate uniformly distributed samples in the cosine of the angle
    cos_theta_samples = np.random.uniform(np.cos(zenith_max_rad), np.cos(zenith_min_rad), n_events)

    # Convert back to zenith angles
    zenith_angles = np.arccos(cos_theta_samples)

    # Generate air shower maximum
    xmax = []
    for i in range(n_events):
      if particle_type =='proton':
        xmax.append(gaussian_random_xmax(energy[i],slope=55,intercept=750,std_deviation=60))
      elif particle_type =='helium':
        xmax.append(gaussian_random_xmax(energy[i],slope=52,intercept=700,std_deviation=50))
      elif particle_type =='oxygen':
        xmax.append(gaussian_random_xmax(energy[i],slope=48,intercept=650,std_deviation=40))
      elif particle_type =='iron':
        xmax.append(gaussian_random_xmax(energy[i],slope=45,intercept=600,std_deviation=30))
      else:
        xmax.append(np.nan)
    #xmax = np.random.normal(loc=800, scale=50, size=n_events)

    # Generate high energy muon number proxy
    muon_number = []
    for i in range(n_events):
      if particle_type =='proton':
        muon_number.append(gaussian_random_with_increasing_mean(energy[i],std_deviation=0.2,slope=0.82,intercept=-3.58))
      elif particle_type =='helium':
        muon_number.append(gaussian_random_with_increasing_mean(energy[i],std_deviation=0.15,slope=0.81,intercept=-3.36))
      elif particle_type =='oxygen':
        muon_number.append(gaussian_random_with_increasing_mean(energy[i],std_deviation=0.10,slope=0.79,intercept=-3.14))
      elif particle_type =='iron':
        muon_number.append(gaussian_random_with_increasing_mean(energy[i],std_deviation=0.10,slope=0.78,intercept=-2.92))
      else:
        muon_number.append(np.nan)

    # Create panda DataFrame
    cosmic_ray_data = pd.DataFrame({
        'Type': ptype,
        'Energy': energy,
        'Log10Energy': np.log10(energy),
        'Zenith': zenith_angles,
        'MuonNumberProxy': muon_number,
        'Xmax': xmax
    })

    return cosmic_ray_data

event_type = {
    "all_raw": ["Log10Energy", "Zenith", "Xmax", "MuonNumberProxy"],
    "energy+zenith": ["Log10Energy", "Zenith"],
    "energy+zenith+xmax": ["Log10Energy", "Zenith", "Xmax"],
    "energy+zenith+muon": ["Log10Energy", "Zenith", "MuonNumberProxy"],
    "all+log": ["Log10Energy", "Zenith", "Xmax", "MuonNumberProxy", "Log_Xmax"],
    "all+ratio": ["Log10Energy", "Zenith", "Xmax", "MuonNumberProxy",\
                  "Energy_Muon_Ratio", "Energy_Xmax_Ratio", "Muon_Xmax_Ratio"],
    "all+product": ["Log10Energy", "Zenith", "Xmax", "MuonNumberProxy",\
                    "Energy_Muon_Product", "Energy_Xmax_Product", "Muon_Xmax_Product"],
    "all": ["Log10Energy", "Zenith", "Xmax", "MuonNumberProxy", \
            "Log_Xmax",\
            "Energy_Muon_Ratio", "Energy_Xmax_Ratio", "Muon_Xmax_Ratio",\
            "Energy_Muon_Product", "Energy_Xmax_Product", "Muon_Xmax_Product"]
}

<font color='Orange' size=5>You can change this value if you want more synthetic data!</font>


In [ ]:
#Define the size of your data sets for proton and iron
num_events = 10000

In [ ]:
# @title Create the syntetic data sets
#Create labeled data sets for proton and iron cosmic rays
df_proton = generate_cosmic_ray_labeled_data(num_events,'proton')
df_helium = generate_cosmic_ray_labeled_data(num_events,'helium')
df_oxygen = generate_cosmic_ray_labeled_data(num_events,'oxygen')
df_iron = generate_cosmic_ray_labeled_data(num_events,'iron')

#Combine labeled datasets to your analysis data set
data = pd.concat([df_proton, df_helium, df_oxygen, df_iron], ignore_index=True)

In [ ]:
# Explore the dataset
print(data.head())  # Check the first few rows
print(" ")
print(data.info())  # Get information about columns and data types
#print(data.describe())  # Statistical summary of numerical columns

In [ ]:
# @title Plot the synthetic data set
# Plotting the measured distributions
fig, ([ax1, ax2], [ax3, ax4]) = plt.subplots(2, 2, figsize=(10, 10))

plot_loghist(df_proton['Energy'],20,ax1)
plot_loghist(df_helium['Energy'],20,ax1)
plot_loghist(df_oxygen['Energy'],20,ax1)
plot_loghist(df_iron['Energy'],20,ax1)
ax1.legend(['Proton','Helium','Oxygen','Iron'])
ax1.set_yscale('log')
ax1.set_xlabel('Energy (GeV)')
ax1.set_ylabel('Number of events')
ax1.set_title('Synthetic Cosmic Ray Energy Distribution')

ax2.hist(df_proton['Zenith'],20,alpha=0.5,label='Proton')
ax2.hist(df_helium['Zenith'],20,alpha=0.5,label='Helium')
ax2.hist(df_oxygen['Zenith'],20,alpha=0.5,label='Oxygen')
ax2.hist(df_iron['Zenith'],20,alpha=0.5,label='Iron')

ax2.legend()
ax2.set_xlabel('Zenith in radians')
ax2.set_ylabel('Number of events')
ax2.set_title('Synthetic Cosmic Ray Zenith Distribution')

ax3.scatter(df_proton['Energy'],df_proton['Xmax'],alpha=0.25,marker='.',label='Proton')
ax3.scatter(df_helium['Energy'],df_helium['Xmax'],alpha=0.25,marker='.',label='Helium')
ax3.scatter(df_oxygen['Energy'],df_oxygen['Xmax'],alpha=0.25,marker='.',label='Oxygen')
ax3.scatter(df_iron['Energy'],df_iron['Xmax'],alpha=0.25,marker='.',label='Iron')
'''
sns.regplot(x=df_proton['Energy'], y=df_proton['Xmax'], ax=ax3, scatter=False, logx=True, label='Proton')
sns.regplot(x=df_helium['Energy'], y=df_helium['Xmax'], ax=ax3, scatter=False, logx=True, label='Helium')
sns.regplot(x=df_oxygen['Energy'], y=df_oxygen['Xmax'], ax=ax3, scatter=False, logx=True, label='Oxygen')
sns.regplot(x=df_iron['Energy'], y=df_iron['Xmax'], ax=ax3, scatter=False, logx=True, label='Iron')
'''

ax3.legend()
ax3.set_title('Synthetic Cosmic Ray Xmax Distribution')

ax3.set_xscale('log')
ax3.set_xlabel('Energy (GeV)')
ax3.set_ylabel(r'Air shower maximum in g/cm$^2$')

ax4.scatter(df_proton['Energy'],df_proton['MuonNumberProxy'],alpha=0.25,marker='.',label='Proton')
ax4.scatter(df_helium['Energy'],df_helium['MuonNumberProxy'],alpha=0.25,marker='.',label='Helium')
ax4.scatter(df_oxygen['Energy'],df_oxygen['MuonNumberProxy'],alpha=0.25,marker='.',label='Oxygen')
ax4.scatter(df_iron['Energy'],df_iron['MuonNumberProxy'],alpha=0.25,marker='.',label='Iron')

ax4.legend()
ax4.set_title('Synthetic Cosmic Ray HE Muon Distribution')
ax4.set_xscale('log')
ax4.set_xlabel('Energy (GeV)')
ax4.set_ylabel('High-energy muon number proxy')

plt.show()

# Feature Engineering

Since cosmic ray data often spans several orders of magnitude, log transformations can be useful.

In [ ]:
data['Log_Xmax'] = np.log(data['Xmax'])

In [ ]:
fig, [ax1,ax2] = plt.subplots(1, 2, figsize=(10, 5))
sns.scatterplot(data=data, x='Energy', y='Log_Xmax', hue='Type',s=5 , ax=ax1)
ax1.set_title('Energy vs Ln(Xmax)')


ax2.set_xlabel('ln(Xmax)')
ax2.set_ylabel('Number of events')
ax2.set_title('Synthetic Cosmic Ray Ln(Xmax) Distribution')
sns.histplot(data=data, x='Log_Xmax', kde=False, ax=ax2,hue='Type')

Creating ratios or interaction terms between features can sometimes capture important relationships. These are just some possible combinations.


In [ ]:
data['Energy_Muon_Ratio']   = data['Log10Energy'] / (data['MuonNumberProxy'] + 1)
data['Energy_Xmax_Ratio']   = data['Log10Energy'] / (data['Xmax'] + 1)
data['Muon_Xmax_Ratio']     = data['MuonNumberProxy'] / (data['Xmax'] + 1)
data['Energy_Muon_Product'] = data['Log10Energy'] * data['MuonNumberProxy']
data['Energy_Xmax_Product'] = data['Log10Energy'] * data['Xmax']
data['Muon_Xmax_Product']   = data['MuonNumberProxy'] * data['Xmax']

In [ ]:
fig, ([ax1, ax2, ax3], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(15, 10))
sns.scatterplot(data=data, x='Log10Energy', y='Energy_Muon_Ratio', hue='Type',s=5 , ax=ax1)
ax1.set_title('Energy_Muon_Ratio vs Log(Energy)')
sns.scatterplot(data=data, x='Log10Energy', y='Energy_Xmax_Ratio', hue='Type',s=5 , ax=ax2)
ax2.set_title('Energy_Xmax_Ratio vs Log(Energy)')
sns.scatterplot(data=data, x='Log10Energy', y='Muon_Xmax_Ratio', hue='Type',s=5 , ax=ax3)
ax3.set_title('Muon_Xmax_Ratio vs MuonNumberProxy')
sns.scatterplot(data=data, x='Xmax', y='Energy_Xmax_Ratio', hue='Type',s=5 , ax=ax4)
ax4.set_title('Energy_Xmax_Ratio vs Xmax')
sns.scatterplot(data=data, x='Log10Energy', y='Energy_Xmax_Product', hue='Type',s=5 , ax=ax5)
ax5.set_title('Energy_Muon_Product vs Log(Energy)')
sns.scatterplot(data=data, x='Log10Energy', y='Muon_Xmax_Product', hue='Type',s=5 , ax=ax6)
ax6.set_title('Energy_Muon_Product vs Log(Energy)')



<font color='Orange' size=5>**Are all observables carrying information about the primary mass?**</font>

<font color='Orange' size=5>**Please select the features you want to train on!**

In [ ]:
trainevents = event_type['energy+zenith']

### Raw features
#event_type['energy+zenith']
#event_type['energy+zenith+xmax']
#event_type['energy+zenith+muon']
#event_type['all_raw']

### Add feature engineering
### Raw features + Logarithmized Xmax
#event_type['all+log']

### Raw features + Ratio
#event_type['all+ratio']

### Raw features + Product
#event_type['all+product']

### Raw features + Logarithmized Xmax + Ratios + Products
#event_type['all']

# Preprocessing of data
Preprocessing of data involves getting your data ready for machine learning algorithms by cleaning, transforming, and organizing it in a format that the model can understand and learn from effectively.

Encoding Categorical Variables:

* If your dataset contains categorical variables (like colors, categories, etc.), you need to convert them into a numerical format because most machine learning algorithms only understand numerical data. One common method is Label Encoding, which assigns a unique number to each category.

Splitting the Data:
* Divide your dataset into two subsets: a training set and a testing set. The training set is used to train the model, while the testing set is used to evaluate its performance. This helps to assess how well the model will generalize to new, unseen data.

Feature Scaling:
* It's important to scale your features to a similar range to prevent some features from dominating others. Standardization and normalization are two common techniques used for feature scaling. Standardization scales the data to have a mean of 0 and a standard deviation of 1, while normalization scales the data to a range between 0 and 1.




In [ ]:
# Preprocessing of the data

# Encode categorical variables if any. In this case transform the name of the particle into a number.
label_encoder = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data['Encoded_'+column] = label_encoder.fit_transform(data[column])

#Drop unused columns
#data = data.drop(columns=['Energy','Type'])

# Split the dataset into features and target variable
X = data.drop(['Encoded_Type'], axis=1)  # Features

y = data['Encoded_Type']                 # Target variable
y_categorical = to_categorical(y)        # Split in categorical shape

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X[trainevents], y_categorical, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Tuning neural network parameter

<font color='Orange' size=4>These are some of the neural network parameters that you can tune and test how they effect your training.</font>

Loss Function:

* It's a measure of how well the model is performing during training.
* Think of it as a way to calculate how "wrong" the model's predictions are compared to the actual values.
* The goal during training is to minimize this "wrongness" or loss.
* Common loss functions include Mean Squared Error (MSE) for regression tasks and Binary Crossentropy or Categorical Crossentropy for classification tasks.

Batch Size:

* It's the number of samples used in one iteration of training.
* Imagine your dataset is a big book. Instead of reading the entire book at once, you read it page by page. Each page represents a batch.
* Using smaller batch sizes helps in faster training and better utilization of memory resources, especially for large datasets.

Number of Epochs:

* An epoch is one complete pass through the entire training dataset.
* The number of epochs defines how many times the model will see the entire dataset during training.
* Just like reading the same book multiple times to remember it better, training for multiple epochs helps the model learn patterns in the data more effectively.
* However, training for too many epochs can lead to overfitting, where the model memorizes the training data instead of learning general patterns.

Dropout:

* It's a regularization technique used to prevent overfitting in neural networks.
* During training, randomly selected neurons (along with their connections) are temporarily ignored or "dropped out."
* It's like temporarily removing certain parts of your brain while studying to ensure that you don't rely too much on specific parts and learn more robustly.
* Dropout helps in improving the generalization ability of the model by forcing it to learn redundant representations of the data, making it more robust to noise and variations.

<font color='Orange' size=5> Please play around with those values! </font>

In [ ]:
loss_function = 'categorical_crossentropy' # 'categorical_crossentropy' 'CategoricalHinge' 'mean_squared_error' or 'mean_absolute_error'
batchsize = 256 # e.g. 32, 64, 128 or 256
num_epoch = 5 # e.g. 5, 10, 20, 30 or 50
dropout = 0.9 # e.g. 0.1, 0.2 or 0.9

The following cell is running the training and testing of the neural network. Depending on the chosen parameters and the model architecture this can take seconds to minutes to complete.

<font color='Orange' size=5>
The model architecture can be changed if you want to! You can add or remove layers.
</font>

In [ ]:
# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),# First Layer of network
    Dropout(dropout),
    BatchNormalization(),
    Dense(64, activation='relu'),# Second Layer of network
    Dropout(dropout),
    Dense(36, activation='relu'),# Third Layer of network
    Dropout(dropout),
    Dense(16, activation='relu'),# Fouth Layer of network
    Dropout(dropout),
    Dense(4, activation='softmax')
])

time_callback = TimePerEpochCallback()

# Compile the model
model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

# Print the model summary
model.summary()



In [ ]:
# Visualize the model architecture
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train,
                    epochs=num_epoch,
                    batch_size=batchsize,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=[time_callback]
                    )
training_histories.append([history,loss_function,batchsize,dropout,num_epoch])
training_features_histories.append(trainevents)
print(" ")
# Model evaluation
# Predictions on the test set
y_pred_proba = model.predict(X_test_scaled)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes))
print('Accuracy:', accuracy_score(y_true_classes, y_pred_classes))

# Confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
#print("Confusion Matrix:")
#print(conf_matrix)

In [ ]:
# @title <font color='Orange' size=5> Plot and compare your training with each other</font>
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
for i, hist in enumerate(training_histories):
  # Plot training history
  ax1.plot(hist[0].history['accuracy'], label=str(training_features_histories[i])+'\n'+'loss_function '+ hist[1] +'\n batchsize ' + str(hist[2]) + '\n dropout ' + str(hist[3]) + '\n epoch number ' + str(hist[4]))
  #plt.plot(hist.history['val_accuracy'], label='val_accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend(prop={'size': 7},bbox_to_anchor=(0., -0.05, 1., 0.),)


for i, hist in enumerate(training_histories):
  ax2.plot(hist[0].history['time_per_epoch'])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Time in seconds')
#ax2.set_yscale('log')

plt.show()

# Confusion matrix
A confusion matrix is a table used in classification tasks to evaluate how well a model is performing in terms of predicting classes. It helps us understand where the model is getting confused or making errors.

Imagine you have a classification task, where you're trying to predict whether a cosmic ray event is proton, helium, oxygen or iron.

Interpretation:
* You want the majority of your predictions to fall on the diagonal (True Positive and True Negative) because those are correct predictions.
* A large number of False Positives and False Negatives indicates areas where the model is making mistakes and may need improvement.
* You can calculate more metrics like accuracy, precision, recall, and F1-score using the values from the confusion matrix to evaluate the model's performance further.

In [ ]:
# @title <font color='Orange' size=5> Plot the confusion matrix for your last training </font>
classes = ['Proton','Helium','Oxygen','Iron']
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Receiver Operating Characteristic (ROC) Curves

In [ ]:
# @title ROC curves
# Compute ROC-AUC for each class
n_classes = y_categorical.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap('tab10', n_classes)

for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors(i), lw=2,
             label=f'Class {label_encoder.inverse_transform([i])[0]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.show()

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
print('ROC-AUC:', roc_auc)

<font color='Orange' size=4> These results show that the classification network is working to determine what kind of primary particle was the cosmic ray but it's by now means perfect! Classification, despite its utility, is inherently imperfect due to the complexity and nuances of real-world data sets. </font>